<a href="https://colab.research.google.com/github/shivam-sunita-puri/TIYA/blob/main/Calculate_average_heart_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
########################################################
# Commands that must be run before using the functions
########################################################

# Substitute YYY below with token received
access_token='YYY'

# Change date below to correct date
url = "https://api.fitbit.com/1/user/-/activities/heart/date/2024-11-05/1d/1sec.json"  # Replace date as needed

# import packages
import pprint
import requests
import time
from datetime import datetime, timedelta
import pandas as pd

# set correct timezone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Australia/Melbourne /etc/localtime
!date

########################################################
#
# calculate_average_heart_rate(start_time, end_time)
# ==================================================
#
# GOAL:
#
# - Calculate average heart rate for the period specified
#
# USAGE:
#
# - Call the function below with start_time and end_time given as strings in HH:MM:SS format (24 hours notation)
#
# - If all the data in the time window is already on the FitBit website, the function returns the avergae heart rate in the time window
#
# - If some of the data still weren't uploaded to FitBit website (data is uploaded every 15-20 minutes), there will be an error message and the function will be returning -99
#
###########################################################
def calculate_average_heart_rate(start_time, end_time):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:
    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objects
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    parsed_start_time = datetime.strptime(start_time, time_format)
    parsed_end_time = datetime.strptime(end_time, time_format)

    # Compare the dates
    if parsed_start_time < df['parsed_time'][0]:
        print(f"start time {parsed_start_time} is earlier than the time stamp of the first data sample {df['parsed_time'].iloc[0]}")
        return -99
    elif parsed_end_time > df['parsed_time'].iloc[-1]:
        print(f"end time {parsed_end_time} is later than the time stamp of the last data sample {df['parsed_time'].iloc[-1]}")
        return -99


    times = [row['time'] for index, row in df.iterrows() if row['parsed_time'] >= parsed_start_time and row['parsed_time'] <= parsed_end_time]
    print(f"DEBUG INFORMATION: {len(times)} time stamps averaged:")
    print(f"DEBUG INFORMATION: {times}")

    values_to_average = [row['value'] for index, row in df.iterrows() if row['parsed_time'] >= parsed_start_time and row['parsed_time'] <= parsed_end_time]
    print(f"DEBUG INFORMATION: {values_to_average}")

    return sum(values_to_average) / len(values_to_average)

  else:
    pprint.pprint(data)

########################################################
#
# average_heart_rate, first_sample_time, last_sample_time = average_last_X_minutes_available(minutes)
# =================================================================
#
# GOAL:
#
# - Get the average heart rate in the last X minutes available
#
# USAGE:
#
# - Call the function below with the duration of the period required in minutes
#
# - It is recommended to monitor last_sample_time to see if the data is up-to-date or not
#
###########################################################

def average_last_X_minutes_available(minutes):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:

    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objectsd
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    last_sample_time = df['time'].iloc[-1]

    X_minutes_before_parsed_time = df['parsed_time'].iloc[-1] - timedelta(minutes=minutes)

    first_sample_parsed_time = next((time_parsed for time_parsed in df['parsed_time'] if time_parsed >= X_minutes_before_parsed_time), None)

    first_sample_time = first_sample_parsed_time.strftime("%H:%M:%S")

    return calculate_average_heart_rate(first_sample_time, last_sample_time), first_sample_time, last_sample_time

  else:
    pprint.pprint(data)

Tue Nov  5 02:28:45 PM AEDT 2024


In [2]:
# Example usage - calculate average in certain time range

rate = calculate_average_heart_rate('09:50:00','09:51:00')

print(rate)

{'errors': [{'errorType': 'invalid_token',
             'message': 'Access token invalid: YYY. Visit '
                        'https://dev.fitbit.com/docs/oauth2 for more '
                        'information on the Fitbit Web API authorization '
                        'process.'}],
 'success': False}
None


In [ ]:
# Example usage - every minute, calculate the average in the last 5 minutes of available data

minutes = 5

while True:
  average_heart_rate, first_sample_time, last_sample_time = average_last_X_minutes_available(minutes)
  print(f"Last available time stamp: {last_sample_time}")
  print(f"Time stamp {minutes} minutes beforehand: {first_sample_time}")
  print(f"Average heart rate in this period: {average_heart_rate}")
  print("")
  time.sleep(60)

DEBUG INFORMATION: 46 time stamps averaged:
DEBUG INFORMATION: ['13:22:30', '13:22:40', '13:22:45', '13:22:50', '13:23:00', '13:23:05', '13:23:10', '13:23:20', '13:23:25', '13:23:30', '13:23:35', '13:23:40', '13:23:45', '13:23:55', '13:24:00', '13:24:10', '13:24:15', '13:24:20', '13:24:25', '13:24:30', '13:24:40', '13:24:45', '13:24:55', '13:25:00', '13:25:05', '13:25:10', '13:25:15', '13:25:20', '13:25:25', '13:25:30', '13:25:40', '13:25:45', '13:25:50', '13:26:00', '13:26:10', '13:26:15', '13:26:20', '13:26:25', '13:26:35', '13:26:40', '13:26:45', '13:26:50', '13:26:55', '13:27:05', '13:27:20', '13:27:25']
DEBUG INFORMATION: [93, 94, 96, 94, 97, 99, 93, 96, 95, 94, 92, 88, 87, 91, 94, 96, 94, 92, 96, 95, 94, 91, 90, 89, 90, 94, 97, 96, 92, 88, 89, 90, 93, 100, 101, 96, 94, 95, 94, 93, 92, 90, 88, 87, 87, 89]
Last available time stamp: 13:27:25
Time stamp 5 minutes beforehand: 13:22:30
Average heart rate in this period: 92.93478260869566



KeyboardInterrupt: 